# Imports

In [4]:
%run ~/REGE/utils.py

# Runs

In [ ]:
files = os.listdir('/home/shafi.z/REGE/data')
data_name = ['citeseer', 'cora', 'coraml', 'polblogs']
attack_type = 'grad'
num_runs = 1
res_dict = {}

for d in data_name:
    
    data_files = ['/home/shafi.z/REGE/data/' + d + '_' + str(ptb) + '.pkl' for ptb in [0.01, 0.1]]
    
    for graph_name in data_files:
        
        # try:
        graph_name = graph_name.replace("/home/shafi.z/REGE/data/", "")
        graph_name = graph_name.replace(".pkl", "")
        print ("============================================")
        print (graph_name) 
        print ("============================================")
        
        res_dict[graph_name] = {}

        pre_attack_dataset = 'citeseer.npz'

        graph_dict = get_graph(graph_name = graph_name,
                               attack_type = attack_type, 
                               pre_attack_dataset = pre_attack_dataset, 
                               data_dir = '/home/shafi.z/REGE/data/', 
                               ensure_connected = False)

        variance_dict = get_variances(graph = graph_dict['graph'],
                                                  dist_mat = None,
                                                  sample_dict = None,
                                                  sample_count = 50,
                                                  eigen_step = 25,
                                                  plot = False, 
                                                  types = ['conformal', 'eigen'], 
                                                  graph_dict = graph_dict)
        
        features = scipy.sparse.csr_array(graph_dict['features'].cpu().numpy())
        labels = graph_dict['labels'].cpu()
        adj = graph_dict['adj']
        idx_train = graph_dict['idx_train']
        idx_test = graph_dict['idx_test']

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        for _ in range(num_runs):
            

            eigen_sampled_var_model, eigen_sampled_var_test, r = train_eigen_sampled_variance_model(graph_dict = graph_dict,
                                                                                                sample_dict = None,
                                                                                                curvature_dict = None,
                                                                                                variance = variance_dict['binary_deviation'],
                                                                                                variance_hyp = 1,
                                                                                                nhid = 16,
                                                                                                lr = 1e-4,
                                                                                                eigen_count = 5,
                                                                                                train_iters = 100,
                                                                                                adjacency = None,
                                                                                                patience = 25)
            try:
                res_dict[graph_name]['rege(d)'].append(eigen_sampled_var_test)
            except: 
                res_dict[graph_name]['rege(d)'] = [eigen_sampled_var_test]
                

            eigen_sampled_var_model, eigen_sampled_var_test, r = train_eigen_sampled_variance_model(graph_dict = graph_dict,
                                                                                                sample_dict = None,
                                                                                                curvature_dict = None,
                                                                                                variance = variance_dict['conformal_radii'],
                                                                                                variance_hyp = 1,
                                                                                                nhid = 16,
                                                                                                lr = 1e-4,
                                                                                                eigen_count = 5,
                                                                                                train_iters = 100,
                                                                                                adjacency = None,
                                                                                                patience = 25)
            try:
                res_dict[graph_name]['rege(m)'].append(eigen_sampled_var_test)
            except: 
                res_dict[graph_name]['rege(m)'] = [eigen_sampled_var_test]


        # except Exception as e: 
        #     print (e)
        #     continue
    

citeseer_0.01
Test set results: loss= 2.1944 accuracy= 0.6672
Epoch :  2770  | Loss :  0.46167603135108954
Early Stopping at Epoch 2770
Teacher Acc :  0.6671698113207547
Student Acc :  0.6728301886792453
